In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Подход 1: градиентный бустинг "в лоб"

1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [3]:
df = pd.read_csv('features.zip', index_col='match_id')
end_game_features = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
         'barracks_status_radiant', 'barracks_status_dire']
y = df['radiant_win']
X = df.drop(end_game_features, axis=1).reset_index().drop('match_id', axis=1)
X

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,1450265551,1,47,4,1706,1198,17,0,1,8,...,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
97226,1450277704,0,43,4,1793,1416,17,0,1,5,...,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0
97227,1450291848,1,98,4,1399,540,1,0,0,5,...,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0
97228,1450292986,1,100,3,1135,766,6,0,2,6,...,0,-21.0,254.0,-85.0,183.0,5,3,3,1,-42.0


2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
missed_columns = [i for i in X.columns if X.count()[i] != len(X)]
missed_columns

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
X.fillna(0, inplace=True)

4. Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
'radiant_win'

'radiant_win'

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [8]:
for i in range(10, 51, 10):
    start_time = time.time()
    cross = cross_val_score(GradientBoostingClassifier(n_estimators=i), X, y, cv=kf, scoring='roc_auc')
    print(f'n_estimators={i}\t roc_auc={cross.mean()}\t time={round(time.time() - start_time, 2)}s')

n_estimators=10	 roc_auc=0.6637173759568478	 time=52.36s
n_estimators=20	 roc_auc=0.681722162516617	 time=95.49s
n_estimators=30	 roc_auc=0.688614862673372	 time=148.78s
n_estimators=40	 roc_auc=0.6934400126999827	 time=185.28s
n_estimators=50	 roc_auc=0.6970232340493301	 time=243.78s


# Подход 2: логистическая регрессия

In [9]:
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [10]:
for i in range(1, 6):
    start_time = time.time()
    cross = cross_val_score(LogisticRegression(penalty='l2', C=i, random_state=0),
                            X, y, cv=kf, scoring='roc_auc')
    print(f'C={i}\t roc_aus={cross.mean()}\t time={round(time.time() - start_time, 2)}s')

C=1	 roc_aus=0.7162236308856719	 time=6.55s
C=2	 roc_aus=0.7162236838996998	 time=4.8s
C=3	 roc_aus=0.7162235101581969	 time=4.3s
C=4	 roc_aus=0.7162234021174452	 time=4.79s
C=5	 roc_aus=0.7162233597533814	 time=5.26s


Лучший параметр для L2-регуляризации - C=2

2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить

In [11]:
categorial = ['lobby_type'] + [f'{j}{i}_hero' for i in range(1, 6) for j in ['r', 'd']]
X.drop(categorial, axis=1, inplace=True)

In [12]:
for i in range(1, 6):
    start_time = time.time()
    cross = cross_val_score(LogisticRegression(penalty='l2', C=i, random_state=0), X, y, cv=kf, scoring='roc_auc')
    print(f'C={i}\t roc_aus={cross.mean()}\t time={round(time.time() - start_time, 2)}s')

C=1	 roc_aus=0.7162860282066865	 time=4.31s
C=2	 roc_aus=0.716285396717854	 time=4.47s
C=3	 roc_aus=0.7162853268716034	 time=4.06s
C=4	 roc_aus=0.7162851174190831	 time=4.22s
C=5	 roc_aus=0.7162853270182273	 time=3.96s


Лучшим параметром для L2-регуляризации без категориальных признаков C=1<br>
Модель стала показывать лучшее качество, но изменения незначительны

3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [13]:
all_heroes = []
for i in categorial[1:]:
    all_heroes += df[i].tolist()
N = len(set(all_heroes))
print(f'Всего в игре существует {N} героев')

Всего в игре существует 108 героев


In [14]:
max_num_hero = max(all_heroes)
print(max_num_hero)

112


Заметим, что герои пронумерованы не от 0 до N, поэтому будем создавать матрицу для мешка слов с shape[1] = максимальному номеру героя. Возможно, что на самом деле героев 112 (или даже ещё больше) но в матчах рассматриваемой выборки были представлены только 108

4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [15]:
X_pick = np.zeros((df.shape[0], max_num_hero))

for i, match_id in enumerate(df.index):
    for p in range(1, 6):
        X_pick[i, df.loc[match_id, f'r{p}_hero'] - 1] = 1
        X_pick[i, df.loc[match_id, f'd{p}_hero'] - 1] = -1
X_pick = pd.DataFrame(X_pick)

In [16]:
X = pd.concat([X, X_pick], axis=1)
X.shape

(97230, 203)

5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [17]:
for i in range(1, 6):
    start_time = time.time()
    cross = cross_val_score(LogisticRegression(penalty='l2', C=i, max_iter=500, random_state=0),
                            X, y, cv=kf, scoring='roc_auc')
    print(f'C={i}\t roc_aus={cross.mean()}\t time={round(time.time() - start_time, 2)}s')

C=1	 roc_aus=0.7517384679466077	 time=17.39s
C=2	 roc_aus=0.7517356762985568	 time=18.5s
C=3	 roc_aus=0.751735784528859	 time=19.64s
C=4	 roc_aus=0.7517359160734604	 time=18.03s
C=5	 roc_aus=0.7517348329747529	 time=17.56s


Лучший параметр C для данных, обработанных с помощью "мешка слов" - C=1

# Обработка тестовых данных и предсказания

In [18]:
df_test = pd.read_csv('features_test.zip', index_col='match_id')
X_test = df_test.fillna(0).reset_index().drop('match_id', axis=1)
X_test[X_test.columns] = scaler.transform(X_test)
X_test.drop(categorial, axis=1, inplace=True)
X_test

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,-2.528200,0.501314,-0.230161,-0.126909,-0.357459,-0.537757,1.017574,0.291758,-0.332256,-0.011207,...,-0.987486,0.478886,1.901284,-0.383372,1.152733,0.018054,0.562864,-0.551154,-0.970428,3.093932
1,-2.527214,-1.297676,-1.195592,-1.245180,-1.131661,-0.537757,-0.578083,0.291758,0.578881,0.008600,...,1.769891,-0.568506,0.835318,1.543833,-1.578386,0.018054,-1.167834,-0.551154,0.437788,0.564676
2,-2.525688,-1.297676,-0.851426,-0.732370,-1.131661,-0.537757,-0.578083,1.893320,-1.243393,-1.383288,...,0.391203,-0.389683,-0.810858,-0.443597,0.289217,-1.030561,-0.302485,0.678170,-0.970428,-0.675155
3,-2.521671,-0.398181,-0.927319,-0.527677,-1.131661,0.968527,1.017574,1.092539,-1.243393,-0.931329,...,1.769891,-1.079429,-1.026750,-0.564048,0.530198,0.018054,0.562864,-0.551154,0.437788,-0.476782
4,-2.520369,0.501314,0.045173,-1.049106,-0.799860,-0.537757,-0.578083,0.291758,1.490017,0.925122,...,-0.987486,1.091994,0.997237,-0.383372,0.228972,-1.030561,-0.302485,-0.551154,0.437788,-0.650358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,1.084410,1.400808,1.448314,1.708865,1.743945,-0.537757,1.017574,-0.108632,0.578881,0.487568,...,0.391203,0.376701,1.982244,-0.443597,-1.578386,0.542361,-0.302485,-0.551154,0.437788,-0.650358
17173,1.086240,-0.398181,-0.856720,-1.169767,-1.131661,-0.537757,-0.578083,1.492930,-0.332256,-0.106640,...,0.391203,0.146786,0.363054,-0.323147,0.269135,-0.506253,-0.302485,1.907494,0.437788,-0.278409
17174,1.086370,-1.297676,-1.162058,-1.398161,-1.242261,-0.537757,-0.578083,-0.108632,-0.332256,-0.886314,...,-0.987486,0.683255,0.363054,-0.503822,0.821384,0.542361,0.562864,1.907494,-0.970428,-0.724748
17175,1.090210,0.501314,1.077673,0.060547,-0.357459,0.968527,2.613231,-0.108632,-0.332256,-0.317314,...,-0.987486,-0.542960,-1.431548,-0.202697,-1.578386,-1.030561,0.562864,-1.780478,-0.970428,0.167930


Проверим не появились ли в тестовой выборке новые герои:

In [19]:
all_test = []
for i in categorial[1:]:
    all_test += df_test[i].tolist()
for i in all_test:
    if not i in all_heroes:
        print('В тестовой выборке есть новые герои!')

Новые герои отсутствуют

In [20]:
max(all_test) == max(all_heroes)

True

Заметим, что максимальный номер героя также совпадает в тестовой и тренировочной выборках

In [21]:
max_num_hero = max(all_heroes)
X_pick_test = np.zeros((df_test.shape[0], max_num_hero))

for i, match_id in enumerate(df_test.index):
    for p in range(1, 6):
        X_pick_test[i, df_test.loc[match_id, f'r{p}_hero'] - 1] = 1
        X_pick_test[i, df_test.loc[match_id, f'd{p}_hero'] - 1] = -1
X_pick_test = pd.DataFrame(X_pick_test)

In [22]:
X_test = pd.concat([X_test, X_pick_test], axis=1)
X_test.shape

(17177, 203)

In [23]:
clf_log = LogisticRegression(penalty='l2', C=1, max_iter=500)
clf_log.fit(X, y);

In [24]:
pred = clf_log.predict_proba(X_test)

In [25]:
for i in [0, 1]:
    print(f'Минимальное значение прогноза на тестовой выборке для класса {i}: {pred[:, i].min()}')
    print(f'Максимальное значение прогноза на тестовой выборке для класса {i}: {pred[:, i].max()}\n')
print(f'Минимальное значение для всех классов: {pred.min()}')
print(f'Максимальное значение для всех классов: {pred.max()}')

Минимальное значение прогноза на тестовой выборке для класса 0: 0.0035684007651590743
Максимальное значение прогноза на тестовой выборке для класса 0: 0.9915429219332391

Минимальное значение прогноза на тестовой выборке для класса 1: 0.0084570780667609
Максимальное значение прогноза на тестовой выборке для класса 1: 0.9964315992348409

Минимальное значение для всех классов: 0.0035684007651590743
Максимальное значение для всех классов: 0.9964315992348409
